In [1]:
import importlib
import data
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

LOADING DATA: 8it [00:37,  3.17s/it]                       

In [2]:
##### Respect du nombre de voies de chantier #####
def retrieve_occupations(OCCUPATIONS):
    # print(len(OCCUPATIONS))
    # print(OCCUPATIONS[0])
    # print(type(OCCUPATIONS[0]))
    # print(OCCUPATIONS[0].getValue())
    chantiers_df = dataframes["chantiers_df"]
    L = {}
    occupation_limits = [chantiers_df[chantiers_df["Chantier"]=="WPY_REC"]["Nombre de voies"].iloc[0],
            chantiers_df[chantiers_df["Chantier"]=="WPY_FOR"]["Nombre de voies"].iloc[0],
            chantiers_df[chantiers_df["Chantier"]=="WPY_DEP"]["Nombre de voies"].iloc[0]]
    for tache_name, sillon_id, line_expr in OCCUPATIONS["DEB"]:
        v = line_expr.getValue()
        
        if v == occupation_limits[0]:
            print("DEB limite atteinte")
        # if v > occupation_limits[0]:
        #     print("DEB limite DEPASSEE")
            
        L[tache_name, sillon_id] = v
        
    for tache_name, sillon_id, line_expr in OCCUPATIONS["FOR"]:
        v = line_expr.getValue()
        # if v == occupation_limits[1]:
        #     print("FOR limite atteinte")
        if v > occupation_limits[1]:
            print("FOR limite DEPASSEE")
        L[tache_name, sillon_id] = v


    for tache_name, sillon_id, line_expr in OCCUPATIONS["DEG"]:
        v = line_expr.getValue()
        # if v == occupation_limits[2]:
        #     print("DEG limite atteinte")
        if v > occupation_limits[2]:
            print("DEG limite DEPASSEE")
        L[tache_name, sillon_id] = v
    return L,occupation_limits


In [3]:
# Excel builder
from openpyxl import Workbook
from datetime import datetime
from utils import min_to_jour

def show_solution(model):
    # print(var_dict)
    print("Objectif minimal: ", m.objVal)
    #build excel of tasks
    workbook = Workbook()
    sheet = workbook.active
    sillons_df = dataframes["sillons_df"]
    taches_df = dataframes["taches_df"]
    machines_df = dataframes["machines_df"]
    L, occupation_limits = retrieve_occupations(OCCUPATIONS)
    occupation_keys = L.keys()

    sheet["A1"] = "Id_tache"
    sheet["B1"] = "Type de tache"
    sheet["C1"] = "Jour"
    sheet["D1"] = "HDébut"
    sheet["E1"] = "Durée"
    sheet["F1"] = "Sillon"
    sheet["G1"] = "Minute"
    sheet["H1"] = "Occupation"
    sheet["I1"] = "Journée de service RAJC"
    row = 2
    for tache in var_dict.keys():
        for sillon_id in var_dict[tache].keys():
            sheet[f"B{row}"] = tache
            sheet[f"A{row}"] = tache + "_" + sillon_id
            sheet[f"C{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[0]
            sheet[f"D{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[1]
            if tache in list(machines_df["Machine"]):
                sheet[f"E{row}"] = machines_df[machines_df["Machine"] == tache]["Duree "].iloc[0]
            elif tache in list(taches_df["Type de tache humaine"]): 
                sheet[f"E{row}"] = taches_df[taches_df["Type de tache humaine"] == tache]["Durée"].iloc[0]
            sheet[f"F{row}"] = sillon_id
            sheet[f"G{row}"] = var_dict[tache][sillon_id].X
            if (tache, sillon_id) in occupation_keys:
                sheet[f"H{row}"] = L[tache, sillon_id]
            for tache_key, sillon_id_key, r,a,j,c in links_dict.keys():
                if tache_key==tache and sillon_id_key==sillon_id and links_dict[tache_key, sillon_id_key, r,a,j,c].X == 1:
                    jds_tsring = f"{r}, {a}, {j}, {c}"
            try:
                sheet[f"I{row}"] = jds_tsring
            except:
                raise Exception("JDS not found, ",tache, sillon_id)
            row += 1
    ##ocupation
    
    occupation_sheet = workbook.create_sheet("Occupation")
    sheet = occupation_sheet
    sheet["A1"] = "Mesure"
    sheet["B1"] = "WPY_REC"
    sheet["C1"] = "WPY_FOR"
    sheet["D1"] = "WPY_DEP"
    
    
    sheet["A2"] = "Taux max occupation"
    sheet["A3"] = "Nb max occupation"
    sheet["A4"] = "Nb max par chantier"
    
    chantier = "WPY_REC"
    occupation_max = max([L["arrivée Reception", sillon_id_key] for tache_name_key, sillon_id_key in L.keys() if tache_name_key == "arrivée Reception" ])
    capacity = occupation_limits[0]
    sheet["B2"] = 100*occupation_max/capacity
    sheet["B3"] = occupation_max
    sheet["B4"] = capacity
    
    chantier = "WPY_FOR"
    occupation_max = max([L["débranchement", sillon_id_key] for tache_name_key, sillon_id_key in L.keys() if tache_name_key == "débranchement" ])
    capacity = occupation_limits[1]
    sheet["C2"] = 100*occupation_max/capacity
    sheet["C3"] = occupation_max
    sheet["C4"] = capacity
    
    chantier = "WPY_DEP"
    
    occupation_max =max([L["dégarage / bouger de rame", sillon_id_key] for tache_name_key, sillon_id_key in L.keys() if tache_name_key == "dégarage / bouger de rame" ])
    capacity = occupation_limits[2]
    sheet["D2"] = 100*occupation_max/capacity
    sheet["D3"] = occupation_max
    sheet["D4"] = capacity

    currentDateAndTime = datetime.now().strftime("%d-%m_%H-%M-%S")
    workbook.save(filename=f"results/{data.NAME}-{currentDateAndTime}.xlsx")
# show_solution(m)


In [4]:
# imports
import variables
import contraintes
import contraintesJ2
import importlib
import utils
importlib.reload(variables) #to remive in final .py

importlib.reload(utils)

<module 'utils' from 'c:\\Users\\remgo\\Desktop\\ProjetST7\\git\\utils.py'>

In [6]:
# pre test J2
importlib.reload(contraintes)

importlib.reload(variables)
importlib.reload(utils)
# create model
m = Model()
# generate variables (integers)
var_dict = variables.generate_variablesJ1(m, dataframes)
links_dict = variables.generate_variablesJ2(m,var_dict,dataframes)
# generate constraints 
contraintes.generate_contraintes(m, dataframes, var_dict)


Initialising variables:   0%|          | 0/217 [00:00<?, ?it/s]

Link variables:   0%|          | 0/5 [00:00<?, ?it/s]

BUILDING CONSTRAINTS:   0%|          | 0/8 [00:00<?, ?it/s]

({(1248, 1248, None, None): <gurobi.Var HELPER 1248,1248>,
  (801, 1248, None, None): <gurobi.Var HELPER 801,1248>,
  (664, 1248, None, None): <gurobi.Var HELPER 664,1248>,
  (1130, 1248, None, None): <gurobi.Var HELPER 1130,1248>,
  (707, 1248, None, None): <gurobi.Var HELPER 707,1248>,
  (1415, 1248, None, None): <gurobi.Var HELPER 1415,1248>,
  (1321, 1248, None, None): <gurobi.Var HELPER 1321,1248>,
  (1429, 1248, None, None): <gurobi.Var HELPER 1429,1248>,
  (1288, 1248, None, None): <gurobi.Var HELPER 1288,1248>,
  (1567, 1248, None, None): <gurobi.Var HELPER 1567,1248>,
  (2793, 1248, None, None): <gurobi.Var HELPER 2793,1248>,
  (1620, 1248, None, None): <gurobi.Var HELPER 1620,1248>,
  (2533, 1248, None, None): <gurobi.Var HELPER 2533,1248>,
  (2058, 1248, None, None): <gurobi.Var HELPER 2058,1248>,
  (2416, 1248, None, None): <gurobi.Var HELPER 2416,1248>,
  (2147, 1248, None, None): <gurobi.Var HELPER 2147,1248>,
  (2203, 1248, None, None): <gurobi.Var HELPER 2203,1248>,
  (

In [7]:
# test J2
importlib.reload(contraintesJ2)
B, OCCUPATIONS, max_horizon = contraintesJ2.generate_contraintes(m, dataframes, var_dict, links_dict)

# deposer solution jalon1  et jalon 2 maybe # new objectif ??
# add excels for jalon 2 in show solution

BUILDING CONSTRAINTS FOR J2:   0%|          | 0/4 [00:00<?, ?it/s]

c:\Users\remgo\Desktop\ProjetST7\git\contraintesJ2.py:70: UserWarning: temp is null for: roulement_reception,1,1,1
  warnings.warn(message=message )
c:\Users\remgo\Desktop\ProjetST7\git\contraintesJ2.py:70: UserWarning: temp is null for: roulement_reception,2,1,1
  warnings.warn(message=message )
c:\Users\remgo\Desktop\ProjetST7\git\contraintesJ2.py:70: UserWarning: temp is null for: roulement_reception,3,1,1
  warnings.warn(message=message )
c:\Users\remgo\Desktop\ProjetST7\git\contraintesJ2.py:70: UserWarning: temp is null for: roulement_reception,4,1,1
  warnings.warn(message=message )
c:\Users\remgo\Desktop\ProjetST7\git\contraintesJ2.py:70: UserWarning: temp is null for: roulement_reception,5,1,1
  warnings.warn(message=message )
c:\Users\remgo\Desktop\ProjetST7\git\contraintesJ2.py:70: UserWarning: temp is null for: roulement_reception,6,1,1
  warnings.warn(message=message )
c:\Users\remgo\Desktop\ProjetST7\git\contraintesJ2.py:70: UserWarning: temp is null for: roulement_formati

In [36]:
# test run gurobi
final_train_task = dataframes["sillons_df"].copy()
final_train_task = final_train_task[final_train_task["LDEP"]=="WPY"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
# final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
# final_train_task = var_dict["essai de frein départ"][final_train_task]
final_train_task = quicksum([var_dict["essai de frein départ"][temp] for temp in list(final_train_task["train_id"])])
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)
# autre idée : maximiser la moyenne de l'ecart entre frein de depart et HDEP
# aute idee minimiser occupation
m.update()
# m.printStats()
# m.write("genconstr.mps")
# m.write("test.lp")
m.optimize()
# m.write("test.lp")


6_41212417/08/202203:1323:5917/08/2022 + 6_43111317/08/202211:0023:5917/08/2022
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 424123 rows, 74972 columns and 1275104 nonzeros
Model fingerprint: 0x6b603ec2
Model has 419101 quadratic constraints
Model has 2580 general constraints
Variable types: 0 continuous, 74972 integer (72602 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [2e+01, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [5e+02, 2e+04]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 383608 rows and 34667 columns
Presolve time: 4.07s
Presolved: 159963 rows, 159753 columns, 466272 nonzeros
Presolved mod

=================================================

In [ ]:
# GO
#to remive in final .py
importlib.reload(utils)
importlib.reload(variables)
importlib.reload(contraintes) 
importlib.reload(contraintesJ2)
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variablesJ1(m, dataframes)
links_dict = variables.generate_variablesJ2(m,var_dict,dataframes)
# generate variables
B, OCCUPATIONS, max_horizon = contraintes.generate_contraintes(m, dataframes, var_dict)
contraintesJ2.generate_contraintes(m, dataframes, var_dict, links_dict)

final_train_task = dataframes["sillons_df"].copy()
final_train_task = final_train_task[final_train_task["LDEP"]=="WPY"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
# final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
# final_train_task = var_dict["essai de frein départ"][final_train_task]
final_train_task = quicksum([var_dict["essai de frein départ"][temp] for temp in list(final_train_task["train_id"])])
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)
# autre idée : maximiser la moyenne de l'ecart entre frein de depart et HDEP
# aute idee minimiser occupation
m.update()
# m.printStats()
# m.write("genconstr.mps")
# m.write("test.lp")
m.optimize()
# m.write("test.lp")


Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [ ]:
show_solution(m)